# Atividade 1 - Etapa 2 - Identificar os períodos de deglutição nos arquivos válidos

In [20]:
import pandas as pd

df = pd.read_csv('arquivos.csv')
df.set_index('0',inplace=True)
df.head()

,1,2,3,4,5,6,7,8,9,10,...,504,505,506,507,508,509,510,511,512,513
0,,,,,,,,,,,,,,,,,,,,,
a00001.mp3,0.619257,0.591173,0.588934,0.576050,0.569039,0.558396,0.591615,0.588218,0.560607,0.597746,...,0.001572,0.001675,0.001099,0.002113,0.002867,0.002273,0.002977,0.001725,0.000856,0.000328
a00002.mp3,0.579076,0.561442,0.574778,0.592735,0.597341,0.586346,0.579085,0.572200,0.559282,0.583274,...,0.005157,0.004259,0.004315,0.004707,0.003548,0.003606,0.003484,0.002382,0.001450,0.000089
a00003.mp3,0.659991,0.666785,0.614244,0.595386,0.595084,0.606930,0.569941,0.548508,0.509913,0.497616,...,0.000201,0.000238,0.000181,0.000245,0.000167,0.000218,0.000177,0.000133,0.000095,0.000008
a00004.mp3,0.595120,0.439680,0.370019,0.396822,0.355503,0.330981,0.379730,0.433511,0.469010,0.576902,...,0.000119,0.000113,0.000272,0.000157,0.000322,0.000315,0.000293,0.000348,0.000121,0.000015
a00005.mp3,0.600767,0.576361,0.621725,0.593066,0.625016,0.594525,0.563292,0.547832,0.565713,0.566939,...,0.004194,0.003753,0.002023,0.003272,0.002138,0.002041,0.001474,0.002276,0.000720,0.000108


## Classificador de Audios Válidos

Classificação geral dos arquivos de audio:

In [21]:
tutor = pd.read_csv('classificacao_anomalias.csv')
tutor = tutor[['File Name','Label']]
tutor['Label'] = tutor['Label'].apply(lambda x: 1 if x == 0 else 0)
tutor.set_index('File Name',inplace=True)
# identificando labels e quantidades
display(tutor['Label'].value_counts())
tutor.head()

Label
0    266
1    224
Name: count, dtype: int64

,Label
File Name,
a00001.mp3,1
a00002.mp3,0
a00003.mp3,1
a00004.mp3,1
a00005.mp3,1


In [22]:
# conectando a tabela de parâmetros da etapa 1 com os sons válidos do tutor
ds = df.join(tutor)
# removo as linhas que não tem identificação de válido
ds.dropna(inplace=True, axis=0)
display(ds.shape)
ds.head()

(490, 514)

,1,2,3,4,5,6,7,8,9,10,...,505,506,507,508,509,510,511,512,513,Label
0,,,,,,,,,,,,,,,,,,,,,
a00001.mp3,0.619257,0.591173,0.588934,0.576050,0.569039,0.558396,0.591615,0.588218,0.560607,0.597746,...,0.001675,0.001099,0.002113,0.002867,0.002273,0.002977,0.001725,0.000856,0.000328,1
a00002.mp3,0.579076,0.561442,0.574778,0.592735,0.597341,0.586346,0.579085,0.572200,0.559282,0.583274,...,0.004259,0.004315,0.004707,0.003548,0.003606,0.003484,0.002382,0.001450,0.000089,0
a00003.mp3,0.659991,0.666785,0.614244,0.595386,0.595084,0.606930,0.569941,0.548508,0.509913,0.497616,...,0.000238,0.000181,0.000245,0.000167,0.000218,0.000177,0.000133,0.000095,0.000008,1
a00004.mp3,0.595120,0.439680,0.370019,0.396822,0.355503,0.330981,0.379730,0.433511,0.469010,0.576902,...,0.000113,0.000272,0.000157,0.000322,0.000315,0.000293,0.000348,0.000121,0.000015,1
a00005.mp3,0.600767,0.576361,0.621725,0.593066,0.625016,0.594525,0.563292,0.547832,0.565713,0.566939,...,0.003753,0.002023,0.003272,0.002138,0.002041,0.001474,0.002276,0.000720,0.000108,1


Utilizando metodologias de aprendizado supervisionado classificador com validação cruzada (CV) para avaliação das técnicas com metrica F1:

In [5]:
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from tqdm import tqdm

In [52]:
X = ds.drop(columns=['Label']).values
y = ds['Label'].values
X.shape, y.shape

((490, 513), (490,))

In [53]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler

Xp = MinMaxScaler().fit_transform(X)
skb = SelectKBest(chi2, k=50)
X = skb.fit_transform(Xp, y)
col_select = skb.get_support(indices=True)
del Xp
X.shape, df.columns[col_select]

((490, 50),
 Index(['19', '44', '53', '56', '57', '58', '59', '60', '61', '62', '63', '64',
        '65', '66', '67', '68', '69', '71', '72', '73', '74', '75', '76', '77',
        '78', '79', '80', '81', '98', '99', '100', '101', '102', '103', '104',
        '105', '106', '109', '233', '249', '253', '258', '270', '271', '273',
        '280', '291', '386', '460', '462'],
       dtype='object'))

In [37]:
import warnings
warnings.filterwarnings('ignore')

max_depth = 5

classificadores = [
    DecisionTreeClassifier(random_state=42,max_depth=max_depth),
    ExtraTreeClassifier(random_state=42,max_depth=max_depth),
    RandomForestClassifier(random_state=42,max_depth=max_depth),
    ExtraTreesClassifier(random_state=42,max_depth=max_depth),
    GradientBoostingClassifier(random_state=42,max_depth=max_depth),
    AdaBoostClassifier(random_state=42),
    HistGradientBoostingClassifier(random_state=42,max_depth=max_depth),
    LogisticRegression(random_state=42),
]

resultados = []
for cls in tqdm(classificadores):
    res = cross_validate(cls, X, y, cv=5, scoring='f1')
    resultados.append(
        {'metodo': cls.__class__.__name__, 
         'f1': res['test_score'].mean(), 
         'tempo': res['fit_time'].mean(),
         }
    )

warnings.filterwarnings('default')
df_res = pd.DataFrame(resultados)
df_res.sort_values('f1', ascending=False)

100%|██████████| 8/8 [00:10<00:00,  1.28s/it]


,metodo,f1,tempo
3,ExtraTreesClassifier,0.634327,0.106601
7,LogisticRegression,0.621525,0.008400
2,RandomForestClassifier,0.609177,0.229191
0,DecisionTreeClassifier,0.605121,0.012204
1,ExtraTreeClassifier,0.598518,0.001004
4,GradientBoostingClassifier,0.557594,1.317599
6,HistGradientBoostingClassifier,0.542417,0.132401
5,AdaBoostClassifier,0.523129,0.199202


In [40]:
import joblib

clf = RandomForestClassifier(random_state=42,max_depth=max_depth)
clf.fit(X, y)

joblib.dump(clf, 'RandomForestClassifier.joblib')

clf.score(X,y)

0.8306122448979592

In [41]:
import joblib
from sklearn.tree import export_text


clf = DecisionTreeClassifier(random_state=42,max_depth=max_depth)
clf.fit(X, y)

joblib.dump(clf, 'DecisionTreeClassifier.joblib')

print('Acurácia:',clf.score(X,y))

print(export_text(clf))

Acurácia: 0.8
|--- feature_29 <= 0.02
|   |--- feature_38 <= 0.00
|   |   |--- feature_0 <= 0.04
|   |   |   |--- feature_34 <= 0.00
|   |   |   |   |--- feature_49 <= 0.00
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_49 >  0.00
|   |   |   |   |   |--- class: 1
|   |   |   |--- feature_34 >  0.00
|   |   |   |   |--- feature_34 <= 0.00
|   |   |   |   |   |--- class: 0
|   |   |   |   |--- feature_34 >  0.00
|   |   |   |   |   |--- class: 1
|   |   |--- feature_0 >  0.04
|   |   |   |--- feature_35 <= 0.00
|   |   |   |   |--- feature_36 <= 0.00
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_36 >  0.00
|   |   |   |   |   |--- class: 0
|   |   |   |--- feature_35 >  0.00
|   |   |   |   |--- class: 0
|   |--- feature_38 >  0.00
|   |   |--- feature_3 <= 0.08
|   |   |   |--- feature_33 <= 0.01
|   |   |   |   |--- feature_28 <= 0.01
|   |   |   |   |   |--- class: 1
|   |   |   |   |--- feature_28 >  0.01
|   |   |   |   |   |--- class: 0
|   |   |  

In [13]:
from IPython.display import Image, Audio, display
from ipywidgets import widgets

ds.reset_index(inplace=True)

blocos = []
for i, row in ds.head(50).iterrows():
    nome = row['0'].split('.')[0]
    out = widgets.Output()
    with out:
        display(Image(f'dataset/Charts/{nome}.png'))
        display('Inválido' if row['Label'] == 0 else 'Válido')
        display(Audio(f'dataset/MP3/{nome}.mp3'))
    blocos.append(out)
widgets.HBox(blocos)